## Presentation Type Classifier (with Hidden Types)

#### Also Network Visualization with Louvain Communities--See below

* **Updated June 2022**
* **Key Features**:

    * Uses `getDistance` to identify `close matches` with side-by-side comparison of soggetti.  
    * For example: With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same. These are reported as "flexed entries" in a separate column.
    * Finds time intervals between entries (expressed as offsets, like `8.0, 4.0, 8.0`)
    * Finds melodic intervals between first note of successive entries in each pattern (like `P-5, P-8`)
    * Labels Fuga, PEn, and ID according to time intervals between entries. NIm not yet supported!
    * If two entries are separated by more than 10 bars (80 offsets), the tool resets to a new pattern
    * Counts number of entries
    * Provides offset and measure/beat locations
    * Sorts all presentation types by the order in which they appear in the piece
    * Reports voice names of the entries, in order of their appearance
    * Omits singleton soggetti (just one entry of a given motive in isolation)

In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


## A.  Start by Finding Presentation Types in a Corpus of Pieces 

- Results are combined into a single dataframe
- Option to save combined results as CSV

In [3]:
prefix = 'https://crimproject.org/mei/'

piece_list = ['CRIM_Model_0019',
             'CRIM_Mass_0019_1',
             'CRIM_Mass_0019_2',
             'CRIM_Mass_0019_3',
             'CRIM_Mass_0019_4',
             'CRIM_Mass_0019_5']

suffix = '.mei'

In [130]:

final = pd.DataFrame()

for piece in piece_list:
    file_url = prefix + piece 
    url = prefix + piece + suffix
    piece = importScore(url)   
    output = piece.presentationTypes(limit_to_entries = True,
                                      edit_distance_threshold = 0,
                                    include_hidden_types = True,
                                    combine_unisons = True,
                                   melodic_ngram_length = 4)

    final['Validation'] = ""
    final['Comments'] = ""
    final = final.append(output, ignore_index=True)
final.head()
# final.to_csv("corpus_classified.csv")
# final.to_csv("saved_csv/CRIM_Mass_0019_P_Types_hidden.csv")

Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.


,Validation,Comments,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[(-3, 3, 2, -2)]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4.0,False
1,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8]","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0]","[(-3, 3, 2, -2)]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0]","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,7.0,False
2,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,40.0,"[6/1.0, 11/3.0, 17/1.0]","[P4, P-4]","[40.0, 84.0, 128.0]","[(-3, 3, 2, -2)]","[44.0, 44.0]","[Tenor, Altus, Bassus]",PEN,3.0,False
3,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,150.0,"[19/4.0, 21/4.0, 22/4.0, 25/4.0, 27/4.0, 29/4....","[P8, P-5, P-4, P-5, P8, P8, P-11]","[150.0, 166.0, 174.0, 198.0, 214.0, 230.0, 250...","[(-3, 2, 2, -2)]","[16.0, 8.0, 24.0, 16.0, 16.0, 20.0, 24.0]","[Tenor, Cantus, Altus, Tenor, Bassus, Altus, C...",FUGA,8.0,False
4,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,150.0,"[19/4.0, 22/4.0, 32/2.0, 35/2.0]","[P4, P8, P-11]","[150.0, 174.0, 250.0, 274.0]","[(-3, 2, 2, -2)]","[24.0, 76.0, 24.0]","[Tenor, Altus, Cantus, Bassus]",ID,4.0,False


In [149]:
# filter for only ID or PEN
pen_filter = final['Presentation_Type'].str.contains("PEN")
pens_only = final[pen_filter]
final = pens_only.copy()



In [150]:
# filter for maximum time interval between entries
final['pen_increment'] = final['Time_Entry_Intervals'].apply(set).apply(sum)
final = final[final['pen_increment'] < 40].copy()


In [151]:
# remove a particular piece
# final = final[final['Title'] != 'Veni sponsa Christi'].copy()


## B. Network Visualization with Louvain Communities

* This will only make sense for a corpus!

## Add Combined Data for Melodic, Time and Soggetti as "Words" for Network Edges

* We combine the lists of time and melodic intervals of entries as strings (thus a kind of word that uniquely identifies that pattern)
* We do the same for soggetti, combining the melodic intervals themselves as a string
* These are in turn assembled in various combinations:  TINT, MINT, and SOG, also as strings

In [142]:
final["MINT"] = final["Melodic_Entry_Intervals"].apply(joiner)
final["TINT"] = final["Time_Entry_Intervals"].apply(joiner)

final['SOG'] = final['Soggetti'].apply(clean_melody_new)
final['ALL'] = final["MINT"] + '_' + final["TINT"] + '_' + final['SOG']

final["ALL_INT"] = final["MINT"] + '_' + final["TINT"]
final["MINT_SOG"] = final["MINT"] + '_' + final["SOG"]
final["TINT_SOG"] = final["TINT"] + '_' + final["SOG"]
final.head()

,Validation,Comments,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,...,Number_Entries,Flexed_Entries,pen_increment,MINT,TINT,SOG,ALL,ALL_INT,MINT_SOG,TINT_SOG
54,,,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Credo,98.0,"[13/2.0, 14/2.0, 17/2.0, 18/2.0]","[P-5, P-4, P8]","[98.0, 106.0, 130.0, 138.0]","[(-2, 2, -2, -2)]","[8.0, 24.0, 8.0]",...,4.0,False,32.0,P-5_P-4_P8,8.0_24.0_8.0,-2_2_-2_-2,P-5_P-4_P8_8.0_24.0_8.0_-2_2_-2_-2,P-5_P-4_P8_8.0_24.0_8.0,P-5_P-4_P8_-2_2_-2_-2,8.0_24.0_8.0_-2_2_-2_-2
55,,,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Credo,178.0,"[23/2.0, 23/4.0, 25/4.0, 26/2.0]","[P-5, P8, P-5]","[178.0, 182.0, 198.0, 202.0]","[(-3, 3, -2, -2)]","[4.0, 16.0, 4.0]",...,4.0,False,20.0,P-5_P8_P-5,4.0_16.0_4.0,-3_3_-2_-2,P-5_P8_P-5_4.0_16.0_4.0_-3_3_-2_-2,P-5_P8_P-5_4.0_16.0_4.0,P-5_P8_P-5_-3_3_-2_-2,4.0_16.0_4.0_-3_3_-2_-2
62,,,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Credo,722.0,"[90/2.0, 90/4.0, 94/2.0, 94/4.0]","[P-5, M-2, P12]","[722.0, 726.0, 754.0, 758.0]","[(4, 2, 2, 2)]","[4.0, 28.0, 4.0]",...,4.0,False,32.0,P-5_M-2_P12,4.0_28.0_4.0,4_2_2_2,P-5_M-2_P12_4.0_28.0_4.0_4_2_2_2,P-5_M-2_P12_4.0_28.0_4.0,P-5_M-2_P12_4_2_2_2,4.0_28.0_4.0_4_2_2_2
63,,,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Credo,722.0,"[90/2.0, 90/4.0, 94/4.0, 95/2.0]","[P-5, P11, P-4]","[722.0, 726.0, 758.0, 762.0]","[(4, 2, 2, 2)]","[4.0, 32.0, 4.0]",...,4.0,False,36.0,P-5_P11_P-4,4.0_32.0_4.0,4_2_2_2,P-5_P11_P-4_4.0_32.0_4.0_4_2_2_2,P-5_P11_P-4_4.0_32.0_4.0,P-5_P11_P-4_4_2_2_2,4.0_32.0_4.0_4_2_2_2
64,,,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Credo,722.0,"[90/2.0, 92/2.0, 93/2.0, 95/2.0]","[P8, M-9, P5]","[722.0, 738.0, 746.0, 762.0]","[(4, 2, 2, 2)]","[16.0, 8.0, 16.0]",...,4.0,False,24.0,P8_M-9_P5,16.0_8.0_16.0,4_2_2_2,P8_M-9_P5_16.0_8.0_16.0_4_2_2_2,P8_M-9_P5_16.0_8.0_16.0,P8_M-9_P5_4_2_2_2,16.0_8.0_16.0_4_2_2_2


### Make Groups, Based on Selected Criteria:

* MINT
* TINT
* SOG
* Or any of the combinations:
    * ALL (MINT + TINT + Soggetti)
    * ALL_INT (MINT + TINT)
    * MINT_SOG (MINT + Soggetti)
    * TINT_SOG (TINT + Soggetti)

In [143]:

output_grouped = final.groupby('Title')['MINT'].apply(list).reset_index()
output_grouped


,Title,MINT
0,Missa Veni sponsa Christi: Agnus Dei,[P-5_P-4_P4]
1,Missa Veni sponsa Christi: Credo,"[P-5_P-4_P8, P-5_P8_P-5, P-5_M-2_P12, P-5_P11_..."
2,Missa Veni sponsa Christi: Sanctus,"[P8_P-11_P8, P-8_P12_P-8, P-8_P4_P5, P-5_P12_P..."


### Group and Find Unique Pairs

- the pairing must be applied to columns that have more than one value, like Title, since these form the edges

In [144]:
pairs = output_grouped.MINT.apply(lambda x: list(combinations(x, 2)))
pairs2 = pairs.explode().dropna()
unique_pairs = pairs.explode().dropna().unique()

In [145]:
unique_pairs

array([('P-5_P-4_P8', 'P-5_P8_P-5'), ('P-5_P-4_P8', 'P-5_M-2_P12'),
       ('P-5_P-4_P8', 'P-5_P11_P-4'), ('P-5_P-4_P8', 'P8_M-9_P5'),
       ('P-5_P-4_P8', 'P12_M-9_P8'), ('P-5_P8_P-5', 'P-5_M-2_P12'),
       ('P-5_P8_P-5', 'P-5_P11_P-4'), ('P-5_P8_P-5', 'P8_M-9_P5'),
       ('P-5_P8_P-5', 'P12_M-9_P8'), ('P-5_M-2_P12', 'P-5_P11_P-4'),
       ('P-5_M-2_P12', 'P8_M-9_P5'), ('P-5_M-2_P12', 'P12_M-9_P8'),
       ('P-5_P11_P-4', 'P8_M-9_P5'), ('P-5_P11_P-4', 'P12_M-9_P8'),
       ('P8_M-9_P5', 'P12_M-9_P8'), ('P8_P-11_P8', 'P-8_P12_P-8'),
       ('P8_P-11_P8', 'P-8_P4_P5'), ('P8_P-11_P8', 'P-5_P12_P-5'),
       ('P8_P-11_P8', 'P8_P-12_P8'), ('P8_P-11_P8', 'P1_P-5_P8'),
       ('P8_P-11_P8', 'P1_P-5_P1'), ('P8_P-11_P8', 'P-5_P8_P-4'),
       ('P-8_P12_P-8', 'P-8_P4_P5'), ('P-8_P12_P-8', 'P-5_P12_P-5'),
       ('P-8_P12_P-8', 'P8_P-12_P8'), ('P-8_P12_P-8', 'P1_P-5_P8'),
       ('P-8_P12_P-8', 'P1_P-5_P1'), ('P-8_P12_P-8', 'P-5_P8_P-4'),
       ('P-8_P4_P5', 'P-5_P12_P-5'), ('P-8_P4_P5', 'P8

In [146]:
pd.Series(unique_pairs).isna().sum()

0

In [147]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G
pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="white", font_color="black")
G = nx.Graph()
G.add_edges_from(unique_pairs)
G = add_communities(G)
pyvis_graph.from_nx(G)
pyvis_graph.show('MINT.html')